# 在 单-三重态量子比特系统中，采用不同脉冲数实现单量子比特任意门的计算结果

In [2]:
import numpy as np
import sys
from numpy import kron
from mindquantum import *
from scipy.linalg import expm
import mindspore as ms
from mindspore import ops
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
from mindspore.nn import Adam, TrainOneStepCell
from mindspore import Tensor
from mindspore.common.parameter import Parameter
from mindspore.common.initializer import initializer  
ms.set_seed(1)
np.random.seed(1)

train_x = np.load('./src/1_qubit_train_x.npy', allow_pickle=True)
eval_x = np.load('./src/1_qubit_eval_x.npy', allow_pickle=True)
train_y = np.load('./src/1_qubit_train_y.npy', allow_pickle=True)
eval_y = np.load('./src/1_qubit_eval_y.npy', allow_pickle=True)
u_mats = np.load('./src/1_qubit_u.npy', allow_pickle=True)

s_x = X.matrix()
s_z = Z.matrix()
one = I.matrix()
dt = np.pi/2

def _matrix_0(coeff):
    return expm(-1j*(coeff*s_z+s_x)*dt)

def _diff_matrix_0(coeff):
    return -1j*_matrix_0(coeff)@(s_z*dt)

gate_0 = gene_univ_parameterized_gate('gete_0', _matrix_0, _diff_matrix_0)

circ = Circuit()
circ += gate_0('00').on(0)
circ += gate_0('01').on(0)
circ += gate_0('02').on(0)
circ += gate_0('03').on(0)
circ += gate_0('04').on(0)
circ += gate_0('05').on(0)
circ += gate_0('06').on(0)
circ += gate_0('07').on(0)
circ += gate_0('08').on(0)
circ += gate_0('09').on(0)

circ += gate_0('10').on(0)
# circ += gate_0('11').on(0)
# circ += gate_0('12').on(0)
# circ += gate_0('13').on(0)
# circ += gate_0('14').on(0)
# circ += gate_0('15').on(0)

ham = Hamiltonian(QubitOperator('')) 
sim = Simulator('projectq', circ.n_qubits)
sim_left = Simulator('projectq',circ.n_qubits)
grad_ops = sim.get_expectation_with_grad(ham,
                                         circ,
                                         circ_left=Circuit(),
                                         simulator_left=sim_left,
                                         ansatz_params_name=circ.params_name)


error_min_list = [] 
m_list = []
lr = 0.1
for i in [10]: #range(len(u_mats)):
    Quantum_net = MQLayer(grad_ops)
    opti = Adam(Quantum_net.trainable_params(), learning_rate=lr)  
    net = TrainOneStepCell(Quantum_net, opti)
    error_min = 1
    for j in range(len(train_x)):
        net(Tensor(train_x[j]), Tensor(train_y[i,j]))
        params = abs(Quantum_net.weight.asnumpy())
        final_state = []
        for k in range(100): # 100 个测试点
            sim.reset()
            sim.set_qs(eval_x[k])
            sim.apply_circuit(circ, params)
            final_state.append(sim.get_qs())
        error = 1-np.real(np.min([np.abs(np.vdot(bra, ket)) for bra, ket in zip(np.array(final_state), eval_y[i])]))
        error_min = min(error, error_min)
        if error_min < 1e-5:
            break
    error_min_list.append(error_min)
    m_list.append(j)
    print(f"进度: {i+1}/{len(u_mats)} ", 'error_min:', error_min, 'iter_num:', j)
    
print('mean of iter_num:', np.mean(m_list))
print('\n mean of error is：', np.mean(error_min_list))
print('\nerror_min_list:\n', error_min_list)

进度: 11/32  error_min: 8.73232940834523e-06 iter_num: 630
mean of iter_num: 630.0

 mean of error is： 8.73232940834523e-06

error_min_list:
 [8.73232940834523e-06]


In [1]:
import numpy as np
import sys
from numpy import kron
from mindquantum import *
from scipy.linalg import expm
import mindspore as ms
from mindspore import ops
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
from mindspore.nn import Adam, TrainOneStepCell
from mindspore import Tensor
from mindspore.common.parameter import Parameter
from mindspore.common.initializer import initializer  
ms.set_seed(1)
np.random.seed(1)

train_x = np.load('./src/1_qubit_train_x.npy', allow_pickle=True)
eval_x = np.load('./src/1_qubit_eval_x.npy', allow_pickle=True)
train_y = np.load('./src/1_qubit_train_y.npy', allow_pickle=True)
eval_y = np.load('./src/1_qubit_eval_y.npy', allow_pickle=True)
u_mats = np.load('./src/1_qubit_u.npy', allow_pickle=True)

s_x = X.matrix()
s_z = Z.matrix()
one = I.matrix()
dt = np.pi/2

def _matrix_0(coeff):
    return expm(-1j*(coeff*s_z+s_x)*dt)

def _diff_matrix_0(coeff):
    return -1j*_matrix_0(coeff)@(s_z*dt)

gate_0 = gene_univ_parameterized_gate('gete_0', _matrix_0, _diff_matrix_0)

circ = Circuit()
circ += gate_0('00').on(0)
circ += gate_0('01').on(0)
circ += gate_0('02').on(0)
circ += gate_0('03').on(0)
circ += gate_0('04').on(0)
circ += gate_0('05').on(0)
circ += gate_0('06').on(0)
circ += gate_0('07').on(0)
circ += gate_0('08').on(0)
circ += gate_0('09').on(0)

circ += gate_0('10').on(0)
# circ += gate_0('11').on(0)
# circ += gate_0('12').on(0)
# circ += gate_0('13').on(0)
# circ += gate_0('14').on(0)
# circ += gate_0('15').on(0)

ham = Hamiltonian(QubitOperator('')) 
sim = Simulator('projectq', circ.n_qubits)
sim_left = Simulator('projectq',circ.n_qubits)
grad_ops = sim.get_expectation_with_grad(ham,
                                         circ,
                                         circ_left=Circuit(),
                                         simulator_left=sim_left,
                                         ansatz_params_name=circ.params_name)


error_min_list = [] 
m_list = []
lr = 0.1
for i in [5]: #range(len(u_mats)):
    Quantum_net = MQLayer(grad_ops)
    opti = Adam(Quantum_net.trainable_params(), learning_rate=lr)  
    net = TrainOneStepCell(Quantum_net, opti)
    error_min = 1
    for j in range(len(train_x)):
        net(Tensor(train_x[j]), Tensor(train_y[i,j]))
        params = abs(Quantum_net.weight.asnumpy())
        final_state = []
        for k in range(100): # 100 个测试点
            sim.reset()
            sim.set_qs(eval_x[k])
            sim.apply_circuit(circ, params)
            final_state.append(sim.get_qs())
        error = 1-np.real(np.min([np.abs(np.vdot(bra, ket)) for bra, ket in zip(np.array(final_state), eval_y[i])]))
        error_min = min(error, error_min)
        if error_min < 1e-5:
            break
    error_min_list.append(error_min)
    m_list.append(j)
    print(f"进度: {i+1}/{len(u_mats)} ", 'error_min:', error_min, 'iter_num:', j)
    
print('mean of iter_num:', np.mean(m_list))
print('\n mean of error is：', np.mean(error_min_list))
print('\nerror_min_list:\n', error_min_list)

进度: 6/32  error_min: 3.7591990563745625e-06 iter_num: 292
mean of iter_num: 292.0

 mean of error is： 3.7591990563745625e-06

error_min_list:
 [3.7591990563745625e-06]
